In [1]:
import pandas as pd
import numpy as np

import pylab as pl
from sklearn.neighbors import KNeighborsClassifier

from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet

import statsmodels.api as sm

### Draft Year Data

In [2]:
def dfplayerYear(year):
    dfplayerYr=pd.read_csv('../Data/draft_year_player/dfplayer'+str(year)+'.csv')
    namePlayer=[]
    
    for elem in dfplayerYr['url']:
        namePlayer.append(elem[11:-5])

    dfplayerYr['name']=namePlayer

    del dfplayerYr['Unnamed: 0']
    del dfplayerYr['url']
    del dfplayerYr['draft']
    del dfplayerYr['shot']
    del dfplayerYr['pos']
    
    return dfplayerYr

In [3]:
fundam = []
for yr in range(1996,2016):
    fundam.append(dfplayerYear(yr))

In [4]:
df_all_fundam = pd.concat(fundam)
df_all_fundam=df_all_fundam.reset_index(drop=True)

df_all_fundam = df_all_fundam[df_all_fundam.height!= 'ml>']
df_all_fundam=df_all_fundam.reset_index(drop=True)
weight_all=df_all_fundam['weight']
height_all=df_all_fundam['height']

df_all_fundam[['weight','height']]=df_all_fundam[['weight','height']].astype(float)

### Height Weight Data

In [9]:
def dfplayerAge(age):
    df_age_yr=pd.read_csv('../Data/AgeData/age_data/df_age_'+str(age)+'.csv',index_col=0)
    return df_age_yr

In [10]:
df_age_frames=[]
for age in range(19,40):
    df_age_frames.append(dfplayerAge(age))

df_ages = pd.concat(df_age_frames)
df_ages=df_ages.reset_index(drop=True)
del df_ages['age_x']


In [11]:
#calculate goals all 
df_ages['goals_all']=df_ages['goals_ev']+df_ages['goals_pp']+df_ages['goals_sh']

#sort by name and age
df_ages_sorted=df_ages.sort(['name', 'age_y'], ascending=[0, 1])

/Users/takehiromatsuzawa/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:5: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [12]:
df_ages_sorted.head()

,TOT,age_y,assists_ev,assists_pp,assists_sh,blocks,corsi_against,corsi_for,corsi_pct,corsi_rel_pct,...,shot_pct,shot_pct.1,shots_attempted,shots_attempted.1,takeaways,time_on_ice_avg,uniqueseason,zs_defense_pct,zs_offense_pct,goals_all
5290,CHI,30.0,2.0,1.0,0.0,35.0,312.0,275.0,46.8,-7.0,...,5.9,5.9,68.0,68.0,6.0,14.58,1,54.2,45.8,2.0
303,MIN,20.0,2.0,0.0,0.0,2.0,73.0,40.0,35.4,-8.8,...,0.0,0.0,16.0,16.0,2.0,11.02,1,48.3,51.7,0.0
730,MIN,21.0,1.0,0.0,0.0,5.0,216.0,183.0,45.9,-6.6,...,11.8,11.8,58.0,58.0,4.0,11.16,1,58.4,41.6,4.0
1330,MIN,22.0,0.0,1.0,0.0,11.0,259.0,198.0,43.3,-5.0,...,10.0,10.0,68.0,68.0,9.0,12.59,1,42.7,57.3,4.0
2030,MIN,23.0,5.0,0.0,0.0,13.0,547.0,641.0,54.0,2.6,...,16.9,16.9,193.0,193.0,21.0,15.04,1,46.2,53.8,21.0


In [13]:
# reset index
df_ages_sorted=df_ages_sorted.reset_index(drop=True)

df_ages_sorted['goals_all']=df_ages_sorted['goals_all']/df_ages_sorted['games_played']
goals_all_data=list(df_ages_sorted['goals_all'])

# you want to predict the next year data
goals_all_data.pop(0)
goals_all_data.append(0)

df_ages_sorted['goals_all']=goals_all_data
df_ages=df_ages_sorted.groupby("name").apply(lambda x: x.iloc[:-1]).reset_index(drop=True)



### Combined Data (All Data)

In [14]:
df_all = pd.merge(df_ages, df_all_fundam, on='name')
df_all=df_all.drop_duplicates()
df_all['assists_all']=df_all['assists_ev']+df_all['assists_pp']+df_all['assists_sh']

df_all['zs_offense_pct']=df_all['zs_offense_pct']/100.0
df_all['shot_pct']=df_all['shot_pct']/100.0

### convert to per game data

In [15]:
per_games=['goals_ev','goals_pp','goals_sh','assists_all','assists_ev','assists_pp','assists_sh','takeaways',
          'hits','giveaways','blocks','shots_attempted']

for per_game in per_games:
    df_all[per_game]=df_all[per_game]/df_all['games_played']

In [16]:
df_all.head()

,TOT,age_y,assists_ev,assists_pp,assists_sh,blocks,corsi_against,corsi_for,corsi_pct,corsi_rel_pct,...,shots_attempted.1,takeaways,time_on_ice_avg,uniqueseason,zs_defense_pct,zs_offense_pct,goals_all,height,weight,assists_all
0,DET,20.0,0.000000,0.0,0.000000,0.000000,18.0,19.0,51.4,-10.0,...,8.0,0.500000,12.13,1,40.0,0.600,0.000000,188.0,218.0,0.000000
1,DET,21.0,0.000000,0.0,0.000000,0.000000,9.0,22.0,71.0,14.6,...,7.0,0.000000,9.18,1,33.3,0.667,0.060000,188.0,218.0,0.000000
2,DET,22.0,0.060000,0.0,0.000000,0.400000,376.0,409.0,52.1,-2.2,...,141.0,0.140000,10.35,1,33.7,0.663,0.094595,188.0,218.0,0.060000
3,DET,23.0,0.148649,0.0,0.013514,0.527027,628.0,765.0,54.9,-0.4,...,226.0,0.229730,12.18,1,41.2,0.588,0.098765,188.0,218.0,0.162162
4,DET,24.0,0.172840,0.0,0.000000,0.518519,698.0,794.0,53.2,-3.2,...,196.0,0.197531,12.19,1,40.7,0.593,0.208333,188.0,218.0,0.172840


### Normalize Data

In [17]:
df_all_nn=df_all[['goals_all','goals_ev','goals_pp','goals_sh','assists_ev','assists_pp','assists_sh','shots_attempted','age_y','blocks',
                 'giveaways','hits','shot_pct','shots_attempted','takeaways','zs_offense_pct','height','weight']]

In [18]:
cols_to_norm = ['goals_ev','goals_pp','goals_sh','assists_ev','assists_pp','assists_sh','shots_attempted','age_y','blocks','giveaways',
               'hits','shot_pct','shots_attempted','takeaways','zs_offense_pct','height','weight']
df_all_nn=df_all_nn[cols_to_norm].apply(lambda x: (x - x.mean()) / (x.max() - x.min()))

In [19]:
df_all_nn.head()

,goals_ev,goals_pp,goals_sh,assists_ev,assists_pp,assists_sh,shots_attempted,shots_attempted,age_y,blocks,giveaways,hits,shot_pct,shots_attempted,shots_attempted,takeaways,zs_offense_pct,height,weight
0,-0.108210,-0.098756,-0.008203,-0.177544,-0.124635,-0.020940,0.091770,0.091770,-0.300227,-0.204734,-0.136415,0.172519,-0.080687,0.091770,0.091770,0.078597,0.092615,0.046712,0.122539
1,-0.108210,-0.098756,-0.008203,-0.177544,-0.124635,-0.020940,0.045937,0.045937,-0.247595,-0.204734,0.030251,0.029662,-0.080687,0.045937,0.045937,-0.214372,0.159615,0.046712,0.122539
2,-0.048210,-0.098756,-0.008203,-0.117544,-0.124635,-0.020940,-0.016395,-0.016395,-0.194964,-0.093073,-0.089749,0.249662,-0.042687,-0.016395,-0.016395,-0.132341,0.155615,0.046712,0.122539
3,-0.013616,-0.098756,-0.008203,-0.028895,-0.124635,0.046627,0.005060,0.005060,-0.142332,-0.057614,-0.046325,0.178311,-0.026687,0.005060,0.005060,-0.079765,0.080615,0.046712,0.122539
4,-0.009445,-0.098756,-0.008203,-0.004704,-0.124635,-0.020940,-0.053083,-0.053083,-0.089700,-0.059989,-0.078802,0.076399,-0.014687,-0.053083,-0.053083,-0.098631,0.085615,0.046712,0.122539


In [20]:
df_all_nn['goals_all']=df_all['goals_all']

In [21]:
df_all_nn['age_y'] = df_all_nn['age_y'].apply(lambda x: abs(x))

In [22]:
df_all_nn=df_all_nn.reset_index(drop=True)

In [23]:
df_all_nn.head()

,goals_ev,goals_pp,goals_sh,assists_ev,assists_pp,assists_sh,shots_attempted,shots_attempted,age_y,blocks,giveaways,hits,shot_pct,shots_attempted,shots_attempted,takeaways,zs_offense_pct,height,weight,goals_all
0,-0.108210,-0.098756,-0.008203,-0.177544,-0.124635,-0.020940,0.091770,0.091770,0.300227,-0.204734,-0.136415,0.172519,-0.080687,0.091770,0.091770,0.078597,0.092615,0.046712,0.122539,0.000000
1,-0.108210,-0.098756,-0.008203,-0.177544,-0.124635,-0.020940,0.045937,0.045937,0.247595,-0.204734,0.030251,0.029662,-0.080687,0.045937,0.045937,-0.214372,0.159615,0.046712,0.122539,0.060000
2,-0.048210,-0.098756,-0.008203,-0.117544,-0.124635,-0.020940,-0.016395,-0.016395,0.194964,-0.093073,-0.089749,0.249662,-0.042687,-0.016395,-0.016395,-0.132341,0.155615,0.046712,0.122539,0.094595
3,-0.013616,-0.098756,-0.008203,-0.028895,-0.124635,0.046627,0.005060,0.005060,0.142332,-0.057614,-0.046325,0.178311,-0.026687,0.005060,0.005060,-0.079765,0.080615,0.046712,0.122539,0.098765
4,-0.009445,-0.098756,-0.008203,-0.004704,-0.124635,-0.020940,-0.053083,-0.053083,0.089700,-0.059989,-0.078802,0.076399,-0.014687,-0.053083,-0.053083,-0.098631,0.085615,0.046712,0.122539,0.208333


### Ordinary Linear Regression

In [24]:

features = ['goals_pp','goals_ev','goals_sh','assists_ev','assists_pp','assists_sh','age_y',
               'hits','shots_attempted','takeaways','zs_offense_pct','weight','height']


results_all_OLR=[]

result=[]
for k in range(0,100):
    test_idx = np.random.uniform(0, 1, len(df_all_nn)) <= 0.8
    train = df_all_nn[test_idx==True]
    test = df_all_nn[test_idx==False]

    train_goals_all=list(train['goals_all'])
    train_data=train[features].as_matrix()
    
    test_data=test[features].as_matrix()
    

    model=sm.OLS(train_goals_all,train_data).fit()
    
    

    preds = model.predict(test_data)
    test_goals_all=list(test['goals_all'])
    distance=0
    for i in range(len(preds)):
        if preds[i]<0:
            distance += pow((test_goals_all[i]), 2)
        else:    
            distance += pow((test_goals_all[i]-preds[i]), 2)


    result.append(distance)
    
    
    
print sum(result)/(len(result)*len(preds))
    

0.0119588459056


### Ridge Regression

In [25]:
alpha_val=[1e-8, 1e-6, 1e-5, 1e-4, 5e-4, 1e-3, 1e-2]
results_all_ridge=[]

for alphaval in alpha_val:
    result=[]
    
    for k in range(0,20):
        test_idx = np.random.uniform(0, 1, len(df_all_nn)) <= 0.8
        train = df_all_nn[test_idx==True]
        test = df_all_nn[test_idx==False]

        train_goals_all=list(train['goals_all'])
        train_data=train[features].as_matrix()
        #train_data=np.array(train_data).tolist()
        test_data=test[features].as_matrix()
        #test_data=np.array(test_data).tolist()

        clfridge = Lasso(alpha=alphaval).fit(train_data,train_goals_all)

        preds = clfridge.predict(test_data)
        test_goals_all=list(test['goals_all'])
        distance=0
        for i in range(len(preds)):
            distance += pow((test_goals_all[i]-preds[i]), 2)

        
        result.append(distance)
    
    results_all_ridge.append([alphaval, sum(result)/(len(result)*len(preds))])
    
print results_all_ridge
    

[[1e-08, 0.0068836376260613032], [1e-06, 0.0078016774057356767], [1e-05, 0.007167111712402684], [0.0001, 0.0069457881116033876], [0.0005, 0.0072819036323297139], [0.001, 0.0072602435677321707], [0.01, 0.016002563751839657]]


In [26]:
results_df_ridge = pd.DataFrame(results_all_ridge, columns=["Penalty", "error"])

pl.plot(results_df_ridge.Penalty, results_df_ridge.error)
pl.title("Error with Increasing Penalty (Ridge)")
pl.xlabel('Penalty')
pl.ylabel('Error')


In [27]:
results_df_ridge

,Penalty,error
0,1.000000e-08,0.006884
1,1.000000e-06,0.007802
2,1.000000e-05,0.007167
3,1.000000e-04,0.006946
4,5.000000e-04,0.007282
5,1.000000e-03,0.007260
6,1.000000e-02,0.016003


### Lasso Regression

In [28]:
alpha_val=[1e-8, 1e-6, 1e-5, 1e-4, 5e-4, 1e-3, 1e-2]

results_all_lasso=[]
dict_coef={}
dict_coef_sum={}
for i in range(len(features)+3):
    dict_coef[i]=0
    dict_coef_sum[i]=0
    
for alphaval in alpha_val:
    result=[]
    
    for k in range(0,100):
        test_idx = np.random.uniform(0, 1, len(df_all_nn)) <= 0.8
        train = df_all_nn[test_idx==True]
        test = df_all_nn[test_idx==False]

        train_goals_all=list(train['goals_all'])
        train_data=train[features].as_matrix()
        
        test_data=test[features].as_matrix()
        
        clflasso = Lasso(alpha=alphaval).fit(train_data,train_goals_all)
        coeffs=clflasso.coef_
        
        for elem in range(len(coeffs)):
            if coeffs[elem]==0:
                dict_coef[elem]=dict_coef[elem]+1
            dict_coef_sum[elem]=dict_coef_sum[elem]+coeffs[elem]/float(800)
            
        preds = clflasso.predict(test_data)
        test_goals_all=list(test['goals_all'])
        distance=0
        for i in range(len(preds)):
            distance += pow((test_goals_all[i]-preds[i]), 2)

        
        result.append(distance)
    
    results_all_lasso.append([alphaval, sum(result)/(len(result)*len(preds))])
    
print results_all_lasso
    

[[1e-08, 0.0068009883589213666], [1e-06, 0.006984048457879], [1e-05, 0.0070216676547142634], [0.0001, 0.0072127912408878345], [0.0005, 0.0068300945908947872], [0.001, 0.0072034247326086976], [0.01, 0.014490902539937421]]


In [29]:
dict_coef_sum

{0: 0.18222165466337226,
 1: 0.3175857487325387,
 2: 0.080940402593123489,
 3: 0.039480642884143775,
 4: -0.005448812785894248,
 5: -0.0090294501377774922,
 6: -0.0034965181291340196,
 7: -0.011338726642180972,
 8: 0.11860894406953232,
 9: 7.7681572574270133e-17,
 10: 2.5290342020146959e-18,
 11: 1.9362693168946242e-18,
 12: 0.091030212511460165,
 13: 0.019289870739275388,
 14: 0.0026824851007206775,
 15: -0.011632005877944876}

In [30]:
dict(zip(train[features].columns, dict_coef_sum.values()))

{'age_y': -0.0034965181291340196,
 'assists_ev': 0.039480642884143775,
 'assists_pp': -0.005448812785894248,
 'assists_sh': -0.0090294501377774922,
 'goals_ev': 0.3175857487325387,
 'goals_pp': 0.18222165466337226,
 'goals_sh': 0.080940402593123489,
 'height': -0.011632005877944876,
 'hits': -0.011338726642180972,
 'shots_attempted': 1.9362693168946242e-18,
 'takeaways': 0.091030212511460165,
 'weight': 0.0026824851007206775,
 'zs_offense_pct': 0.019289870739275388}

In [31]:
dict_coef.values()

[0, 100, 318, 100, 358, 396, 347, 283, 100, 343, 444, 450, 100, 300, 378, 266]

In [32]:
results_df_lasso = pd.DataFrame(results_all_lasso, columns=["Penalty", "error"])

pl.plot(results_df_lasso.Penalty, results_df_lasso.error)
pl.title("Error with Increasing Penalty (Lasso)")
pl.xlabel('Penalty')
pl.ylabel('Error')
